<a href="https://colab.research.google.com/github/Isafon/ECE528/blob/main/ECE528_ASN2_Q1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Q1 Part A ECE528 LA2 - Isa Fontana

## Load, Split, Normalize

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# load
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()


val_size = 10000
x_val, y_val = x_train[-val_size:], y_train[-val_size:]
x_train, y_train = x_train[:-val_size], y_train[:-val_size]


x_train = (x_train.astype("float32") / 255.0)[..., None]
x_val   = (x_val.astype("float32")   / 255.0)[..., None]
x_test  = (x_test.astype("float32")  / 255.0)[..., None]

print(x_train.shape, x_val.shape, x_test.shape)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(50000, 28, 28, 1) (10000, 28, 28, 1) (10000, 28, 28, 1)


## Build a lecture-style CNN

In [2]:
tf.random.set_seed(0)

model = keras.Sequential([
    keras.layers.Input((28, 28, 1)),
    keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(128, 3, padding="same", activation="relu"),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(10, activation="softmax"),
], name="fmnist_cnn")

model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "fmnist_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 241,546 (943.54 KB)

 Trainable params: 241,546 (943.54 KB)

 Non-trainable params: 0 (0.00 B)

## Train

In [3]:
callbacks = [
    keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=2, min_lr=1e-5, verbose=1),
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True, verbose=1),
]

history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=15,
    batch_size=128,
    callbacks=callbacks,
    verbose=2
)

Epoch 1/15
391/391 - 15s - 38ms/step - accuracy: 0.7809 - loss: 0.5976 - val_accuracy: 0.8637 - val_loss: 0.3809 - learning_rate: 1.0000e-03
Epoch 2/15
391/391 - 1s - 3ms/step - accuracy: 0.8700 - loss: 0.3566 - val_accuracy: 0.8834 - val_loss: 0.3153 - learning_rate: 1.0000e-03
Epoch 3/15
391/391 - 1s - 3ms/step - accuracy: 0.8899 - loss: 0.3038 - val_accuracy: 0.8972 - val_loss: 0.2818 - learning_rate: 1.0000e-03
Epoch 4/15
391/391 - 1s - 3ms/step - accuracy: 0.9005 - loss: 0.2714 - val_accuracy: 0.9018 - val_loss: 0.2673 - learning_rate: 1.0000e-03
Epoch 5/15
391/391 - 1s - 3ms/step - accuracy: 0.9081 - loss: 0.2513 - val_accuracy: 0.9093 - val_loss: 0.2427 - learning_rate: 1.0000e-03
Epoch 6/15
391/391 - 1s - 3ms/step - accuracy: 0.9145 - loss: 0.2322 - val_accuracy: 0.9073 - val_loss: 0.2546 - learning_rate: 1.0000e-03
Epoch 7/15
391/391 - 1s - 3ms/step - accuracy: 0.9201 - loss: 0.2157 - val_accuracy: 0.9134 - val_loss: 0.2383 - learning_rate: 1.0000e-03
Epoch 8/15
391/391 - 1s -

## Evaluate Test Set

In [4]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

Test accuracy: 0.9223
